In [38]:
import tensorflow as tf
import tensorflow_datasets as tfds

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(image, tf.complex64) / 255., label

ds_train = ds_train.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [79]:
from tensorflow.keras import datasets, layers, models
import cvnn.initializers as initializers
from cvnn.activation_functions import apply_activation
import numpy as np
from pdb import set_trace

class ComplexDense(layers.Layer):
    def __init__(self, units, activation=None, weight_initializer=None, bias_initializer=None, dtype=np.complex64, **kwargs):
        super(ComplexDense, self).__init__(dtype=dtype, **kwargs)
        if activation is None:
            activation = 'linear'
        self.activation = activation
        if weight_initializer is None:
            weight_initializer = initializers.GlorotUniform()
        self.weight_initializer = weight_initializer
        if bias_initializer is None:
            bias_initializer = initializers.Zeros()
        self.bias_initializer = bias_initializer
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
                                 initializer=self.weight_initializer,
                                 trainable=True,
                                 shape=[int(input_shape[-1]), self.units]
                                )
        self.b = self.add_weight(
            shape=(self.units,), initializer=self.bias_initializer, trainable=True
        )

    def call(self, inputs, training=None):
        print(inputs.dtype)
        out = tf.add(tf.matmul(inputs, self.w), self.b)
        y_out = apply_activation(self.activation, out)
        return y_out

In [76]:
class ComplexFlatten(layers.Flatten):
    def __init__(self, dtype=np.complex64, **kwargs):
        super(ComplexFlatten, self).__init__(dtype=dtype, **kwargs)

In [80]:
model = tf.keras.models.Sequential([
  # ComplexFlatten(dtype=tf.complex64, input_shape=(28, 28, 1)),
  ComplexDense(128, activation='cart_relu', dtype=np.complex64, input_shape=(28, 28, 1)),
  # ComplexDense(10, activation='softmax_real', dtype=np.complex64)
])
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

<dtype: 'complex64'>


TypeError: Value passed to parameter 'x' has DataType complex64 not in list of allowed values: bfloat16, float16, float32, float64, int32, int64

In [33]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_12 (Flatten)         (None, 784)               0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [36]:
model.predict(ds_train)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [35]:
model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6


469/469 [==============================] - 14s 30ms/step - loss: 20.6895 - accuracy: 0.0000e+00 - val_loss: 20.7047 - val_accuracy: 0.0000e+00
Epoch 2/6
469/469 [==============================] - 3s 6ms/step - loss: 20.6895 - accuracy: 0.0000e+00 - val_loss: 20.7047 - val_accuracy: 0.0000e+00
Epoch 3/6
469/469 [==============================] - 3s 7ms/step - loss: 20.6895 - accuracy: 0.0000e+00 - val_loss: 20.7047 - val_accuracy: 0.0000e+00
Epoch 4/6
469/469 [==============================] - 4s 9ms/step - loss: 20.6895 - accuracy: 0.0000e+00 - val_loss: 20.7047 - val_accuracy: 0.0000e+00
Epoch 5/6
469/469 [==============================] - 3s 7ms/step - loss: 20.6895 - accuracy: 0.0000e+00 - val_loss: 20.7047 - val_accuracy: 0.0000e+00
Epoch 6/6
469/469 [==============================] - 3s 7ms/step - loss: 20.6895 - accuracy: 0.0000e+00 - val_loss: 20.7047 - val_accuracy: 0.0000e+00
